In [1]:
import keras
import re
import nltk
import numpy as np
from keras import preprocessing
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import SeparableConv1D, MaxPooling1D
from keras.layers import LSTM, GRU
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint


keras.__version__

Using TensorFlow backend.


'2.2.4'

In [3]:
trainFromTextFile = "train.FROM"
trainToTextFile   = "train.TO"
trainFromText     = open(trainFromTextFile, 'r', encoding='utf-8').read().lower()
trainToText       = open(trainToTextFile, 'r', encoding='utf-8').read().lower()
trainFromSentenceTokens = re.split('\n', trainFromText)
trainToSentenceTokens   = re.split('\n', trainToText)
trainFromWordTokens = re.split(' |\n', trainFromText)
trainToWordTokens   = re.split(' |\n', trainToText)
print(trainFromSentenceTokens[1])
print(trainToSentenceTokens[1])
print('Found %s sentences from TrainFrom Text' %len(trainFromSentenceTokens))
print('Found %s sentences from TrainTo Text' %len(trainToSentenceTokens))
print('Found %s words from TrainFrom Text' %len(trainFromWordTokens))
print('Found %s words from TrainTo Text' %len(trainToWordTokens))

["is it me, or does 'frank moss' look like a child molester?", 'why wait that they finish college or university?  you do know that those are free of charge in most european countries, right?', "or if you christians got off of little boys' asses...", "'undoubtedly'?  some would disagree and say jesus was a good jew, albeit a reformer, and that st. paul actually started christianity.   newlinechar  newlinechar  newlinechar  newlinechar just thought i'd throw that out there."]
["to me, it looks more like he 'takes a wide stance' on public restroom sex.", 'are they for foreign nationals? i know uk universities charge fees of around £10,000 a year for foreign students (the uk students pay around £3,000 a year plus taxation). even when the uk system was free for uk nationals i still think we charged non-eu members to join.', "i'm sure there's never been an atheist pederast...ever.", "there are others who believe that the christian church didn't start until much later and invented several cen

In [7]:
train_len = 100
text_sequences = []
for i in range(train_len,len(trainFromWordTokens)):
    seq = trainFromWordTokens[i-train_len:i]
    text_sequences.append(seq)

print('Found %s words.' %len(trainFromWordTokens))
print('Found %s sequences.' %len(text_sequences))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

data = pad_sequences(sequences, maxlen=train_len)

print(len(sequences))

word_index = tokenizer.word_index
word_count = tokenizer.word_counts
nWords     = len(tokenizer.word_counts) + 1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

    
print('Found %s unique tokens.' % len(word_index))
print('Found %s unique words.' % len(word_count))


Found 521666 words.
Found 521566 sequences.
521566
Found 59671 unique tokens.
Found 59671 unique words.


In [10]:
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(trainFromSentenceTokens)
sequences2 = tokenizer.texts_to_sequences(trainFromSentenceTokens)

sequences2 = pad_sequences(sequences2, maxlen=train_len)

word_index2 = tokenizer2.word_index
word_count2 = tokenizer2.word_counts
nWords     = len(tokenizer2.word_counts) + 1

n_sequences2 = np.empty([len(sequences2),train_len], dtype='int32')
for i in range(len(sequences2)):
    n_sequences2[i] = sequences2[i]

    
print('Found %s unique tokens.' % len(word_index2))
print('Found %s unique words.' % len(word_count2))

Found 34823 unique tokens.
Found 34823 unique words.


In [11]:
print(len(sequences2))
print(len(n_sequences2))

29620
29620


In [16]:
sequences2[10]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   193,    30,     2, 20176,     8, 20177,
         337,    17, 20178,     7,    95,   337,    71,    52,     4,
          18,   496,   676,  4997,     8,  5716,    17,   133,  3414,
         119])

In [17]:
n_sequences2[10]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   193,    30,     2, 20176,     8, 20177,
         337,    17, 20178,     7,    95,   337,    71,    52,     4,
          18,   496,   676,  4997,     8,  5716,    17,   133,  3414,
         119])

In [27]:
train_len = 100

print('Found %s words.' %len(trainFromWordTokens))
print('Found %s sequences.' %len(trainFromSentenceTokens))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainFromSentenceTokens)
sequences = tokenizer.texts_to_sequences(trainFromSentenceTokens)

sequences = pad_sequences(sequences)

print('Found %s.' %len(sequences))

word_index = tokenizer.word_index
word_count = tokenizer.word_counts
nWords     = len(tokenizer.word_counts) + 1

    
print('Found %s unique tokens.' % len(word_index))
print('Found %s unique words.' % len(word_count))

Found 521666 words.
Found 29620 sequences.
Found 29620.
Found 34823 unique tokens.
Found 34823 unique words.


In [25]:
sequences[10]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,   228,    29,     1, 15833,     7, 15834,   363,
          15, 15835,     6, 15836,   111,   363,    65,    58,     3,
          18,   415,   651,  4980,     7,  3587,    15,   151,  3156,
          83])

In [26]:
trainFromSentenceTokens[10]

'seems like the wildest and craziest guys are pastors of megachurches.  those guys know how to have fun.  republican senators and representatives are pretty wild too.'